# Best Performing notebook from Kaggle

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e5/sample_submission.csv
/kaggle/input/playground-series-s5e5/train.csv
/kaggle/input/playground-series-s5e5/test.csv


In [2]:
df = pd.read_csv("/kaggle/input/playground-series-s5e5/train.csv")

# Weight_per_Age
df['Weight_per_Age'] = df['Weight'] / (df['Age'] + 1)

# HeartRate per Weight
df['HeartRate_per_kg'] = df['Heart_Rate'] / df['Weight']

# Duration Per Age
df['Duration_per_age'] = df['Duration'] / (df['Age'] + 1)

# Duration * Heart Rate
df['Duration_heart_rate']=df['Duration']*df['Heart_Rate']

# Intensity
df['Duration_per_weight']=df['Duration']/df['Weight']

# All Durations add and multi
df['duration_sum']=df['Duration_per_weight']+df['Duration_heart_rate']+df['Duration_per_age']
df['duration_multi']=df['Duration_per_weight']*df['Duration_heart_rate']*df['Duration_per_age']

# Creating new column 'BMI'
df['BMI']=df['Weight']/((df['Height']/100) ** 2)
df['BMI']=df['BMI'].round(2)

In [ ]:
from sklearn.neural_network import MLPRegressor
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error
from sklearn.preprocessing import StandardScaler,  OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import GradientBoostingRegressor



X = df.drop(columns=["id", "Calories"])

y = np.log(df['Calories'])


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)



numerical_features = [col for col in X.columns if col not in ["Sex"]]

categorical_features = ["Sex"]


# Create a ColumnTransformer to apply different preprocessing strategies

preprocessor = ColumnTransformer(

    transformers=[

        ("num", StandardScaler(), numerical_features),

        ("cat", OneHotEncoder(), categorical_features),

    ],

).fit(X)



X_train = preprocessor.transform(X)

X_test = preprocessor.transform(X_test)


# Define base models

base_models = [

    ('xgb', XGBRegressor(n_estimators=5000, learning_rate=0.061849546072614363, max_depth=5)),

    ('lgbm', LGBMRegressor(n_estimators=3000, learning_rate=0.021849546072614363, max_depth=16)),

    ('cat', CatBoostRegressor(learning_rate=0.06758463422, max_depth=11, iterations =370, verbose=0)),
    


]


# Define meta-model (aggregator)
meta_model = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42)


# Build Stacking Ensemble

stacking_ensemble = StackingRegressor(
    estimators=base_models, final_estimator=meta_model)

stacking_ensemble.fit(X_train, y)


# Predictions

y_pred = stacking_ensemble.predict(X_test)


rmsle_score = mean_squared_log_error(np.exp(y_test), np.exp(y_pred))
print(f"Optimized Stacking Ensemble RMSLE: {rmsle_score:.4f}")
r2 = r2_score(np.exp(y_test), np.exp(y_pred))
print(f"R-squared Score: {r2}")

2025-05-09 16:51:15.535856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746809475.832443      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746809475.918509      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.062068 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2400
[LightGBM] [Info] Number of data points in the train set: 750000, number of used features: 16
[LightGBM] [Info] Start training from score 4.111361
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.046413 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2401
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 16
[LightGBM] [Info] Start training from score 4.111761
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.045154 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2398
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 16
[LightGBM] [Info] Start 

In [4]:
X_submission = pd.read_csv("/kaggle/input/playground-series-s5e5/test.csv")

# Weight_per_Age
X_submission['Weight_per_Age'] = X_submission['Weight'] / (X_submission['Age'] + 1)

# HeartRate per Weight
X_submission['HeartRate_per_kg'] = X_submission['Heart_Rate'] / X_submission['Weight']

# Duration Per Age
X_submission['Duration_per_age'] = X_submission['Duration'] / (X_submission['Age'] + 1)

# Duration * Heart Rate
X_submission['Duration_heart_rate']=X_submission['Duration']*X_submission['Heart_Rate']

# Intensity
X_submission['Duration_per_weight']=X_submission['Duration']/X_submission['Weight']

# All Durations add and multi
X_submission['duration_sum']=X_submission['Duration_per_weight']+X_submission['Duration_heart_rate']+X_submission['Duration_per_age']
X_submission['duration_multi']=X_submission['Duration_per_weight']*X_submission['Duration_heart_rate']*X_submission['Duration_per_age']

# Creating new column 'BMI'
X_submission['BMI']=X_submission['Weight']/((X_submission['Height']/100) ** 2)
X_submission['BMI']=X_submission['BMI'].round(2)

out=X_submission[["id"]]
X_submission = preprocessor.transform(X_submission.drop(columns=["id"]))
#X_submission = poly.transform(X_submission)
#X_submission = pca.transform(X_submission)
y_out =  stacking_ensemble.predict(X_submission)
out["Calories"] = np.exp(y_out)
out.to_csv("submission.csv", index=False)
out

,id,Calories
0,750000,27.162572
1,750001,108.856540
2,750002,87.529183
3,750003,125.369641
4,750004,75.791915
...,...,...
249995,999995,25.912894
249996,999996,9.668233
249997,999997,73.055768
249998,999998,169.331192
